In [ ]:
# News API

from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='78e67572b7b14c8780a04441f90d91d2')

all_articles = newsapi.get_everything(q='Salzburg',
                                      sources=r'bbc-news,financial-times,the-guardian-uk, the-guardian-au\
                                      , reuters, bloomberg, the-new-york-times, the-washington-post',
                                      #domains='bbc.co.uk,techcrunch.com',
                                      from_param='2018-09-02',
                                      to='2018-10-01',
                                      language='en',
                                      sort_by='relevancy',
                                      page_size=100,
                                      page=3)

df_articles = pd.DataFrame.from_dict(all_articles)

total = df_articles['totalResults'].values[0]

lst_articles = df_articles['articles'].values

df_articles =pd.DataFrame.from_records(lst_articles)

df_articles

## Anyway, we are only interested in counts 

import datetime
t = datetime.datetime(2012, 2, 23, 0, 0)
t.strftime('%Y-%m-%d')

def create_weeks(today):
    res = []
    start = datetime.date.today()- datetime.timedelta(days=30)
    week_number = 0
    while True:
        start_week = start - datetime.timedelta(days=start.weekday())
        end_week = start_week + datetime.timedelta(days=6)
        start = end_week + datetime.timedelta(days=1)
        week_number += 1
        if end_week >= today:
            end_week = today
            res.append((week_number, start_week, end_week))
            return res
        res.append((week_number, start_week, end_week))

weeks = create_weeks(datetime.date.today())

def get_articles_count(start_date, end_date, keyword):
    try:
        all_articles = newsapi.get_everything(q=keyword,
                                      sources=r'bbc-news,financial-times,the-guardian-uk, the-guardian-au\
                                      , reuters, bloomberg, the-new-york-times, the-washington-post',
                                      #domains='bbc.co.uk,techcrunch.com',
                                      from_param=start_date.strftime('%Y-%m-%d'),
                                      to=end_date.strftime('%Y-%m-%d'),
                                      language='en',
                                      sort_by='relevancy',
                                      page_size=100,
                                      page=1)
        if len(all_articles) > 0:
            df_articles = pd.DataFrame.from_dict(all_articles)

            return df_articles['totalResults'].values[0]
    except:
        return 0


kw_counted_p_week = []
for week in weeks:
    for k in keywords:
        cnt = 0
        count = get_articles_count(week[1], week[2], k)
        res_dict = {'week number':week[0], 'start':week[1].strftime('%Y-%m-%d'), 'end':week[2].strftime('%Y-%m-%d'),'keyword':k, 'count': count }
        if len(kw_counted_p_week) == 0:
            kw_counted_p_week = [res_dict]
        else:
            kw_counted_p_week.append(res_dict)
            
df_counted_kw = pd.DataFrame.from_dict(kw_counted_p_week)

df_counted_kw.to_csv(r'D:\FuturesGroup\Contextuality\weaponization data\' + kw_counted_p_week.csv)